Step-by-step plan

1. Find a file named like `conetic-files.txt` in the provided directory.
2. Read and normalize lines, removing empties and duplicates while preserving order.
3. Load `conetic-dump-locations.csv` (searched for in the same directory if not given explicitly) and build a mapping of data file -> dump location.
4. Produce two scripts in the target directory:
   - `001-HERC1-Export.sh` contains pull commands for files NOT present in the CSV mapping.
   - `002-HERC2-Import.sh` contains clearlf/put commands. If a mapping exists, use the mapped dump file; otherwise use the exported CSV from `001-HERC1-Export.sh`.
5. Make the scripts executable.

Usage: call `generate_herc_scripts(target_dir)` from the notebook or another script.

In [4]:
import os
import csv
from collections import OrderedDict

def _find_file_in_dir(target_dir, name_patterns):
    """Return first matching filepath in target_dir for any pattern in name_patterns."""
    for pat in name_patterns:
        p = os.path.join(target_dir, pat)
        if os.path.isfile(p):
            return p
    # fallback: glob-like search for files that contain the name
    for entry in os.listdir(target_dir):
        for pat in name_patterns:
            if pat in entry and os.path.isfile(os.path.join(target_dir, entry)):
                return os.path.join(target_dir, entry)
    return None

def generate_herc_scripts(target_dir,
                         conetic_filename='conetic-files.txt',
                         dumps_csv_name='conetic-dump-locations.csv'):
    """Generate 001-HERC1-Export.sh and 002-HERC2-Import.sh in target_dir.
    - Reads conetic-files.txt (searched for if not exactly present)
    - Cross references conetic-dump-locations.csv (prefers the CSV in the same directory as this notebook/file)
    """
    target_dir = os.path.abspath(target_dir)
    # locate conetic-files
    conetic_path = _find_file_in_dir(target_dir, [conetic_filename])
    if not conetic_path:
        raise FileNotFoundError(f"Could not find {conetic_filename} in {target_dir}")

    # read lines and dedupe while preserving order
    with open(conetic_path, 'r', encoding='utf-8', errors='ignore') as f:
        raw_lines = [ln.strip() for ln in f if ln.strip()]
    # preserve order
    unique_paths = list(OrderedDict.fromkeys(raw_lines))

    # Prefer the dumps CSV located in the same directory as this notebook/script.
    dumps_csv_path = None
    try:
        # when run as a script this gives the script dir; in a notebook __file__ is undefined
        nb_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        # in a Jupyter session the current working directory is typically the notebook dir
        nb_dir = os.getcwd()
    candidate = os.path.join(nb_dir, dumps_csv_name)
    if os.path.isfile(candidate):
        dumps_csv_path = candidate
    else:
        # fallback: look in the target_dir or current working dir
        dumps_csv_path = _find_file_in_dir(target_dir, [dumps_csv_name])
        if dumps_csv_path is None and os.path.isfile(dumps_csv_name):
            dumps_csv_path = os.path.abspath(dumps_csv_name)

    mapping = {}
    if dumps_csv_path and os.path.isfile(dumps_csv_path):
        with open(dumps_csv_path, newline='', encoding='utf-8', errors='ignore') as csvfile:
            try:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    # guard against malformed rows
                    key = (row.get('conetic_data_file') or '').strip()
                    val = (row.get('dump_location') or '').strip()
                    if key:
                        mapping[key] = val
            except Exception:
                # fallback: manual parse if header missing
                csvfile.seek(0)
                for line in csvfile:
                    parts = [p.strip() for p in line.split(',', 1)]
                    if len(parts) == 2:
                        mapping[parts[0]] = parts[1]

    export_path = os.path.join(target_dir, '001-HERC1-Export.sh')
    import_path = os.path.join(target_dir, '002-HERC2-Import.sh')

    with open(export_path, 'w', encoding='utf-8') as expf, open(import_path, 'w', encoding='utf-8') as impf:
        # headers
        expf.write('#!/bin/sh\n')
        expf.write('# Generated export script\n\n')
        impf.write('#!/bin/sh\n')
        impf.write('# Generated import script\n\n')

        for data_file in unique_paths:
            data_file = data_file.strip()
            if not data_file:
                continue
            dump_location = mapping.get(data_file)

            data_file_name = os.path.basename(data_file)
            if not data_file_name:
                # fallback to a safe name
                data_file_name = data_file.replace('/', '_').replace('\\\\', '_').strip('_')
            exported_csv_ref = f"{target_dir}/{data_file_name}.csv"

            if dump_location:
                # write only import lines using mapped dump location
                impf.write(f"clearlf {data_file}\n")
                impf.write(f"put -f {data_file} < {dump_location}\n\n")
            else:
                # write export command to export script
                expf.write(f"pull -af {data_file} > {exported_csv_ref}\n")
                # write import commands that will use the exported csv
                impf.write(f"clearlf {data_file}\n")
                impf.write(f"put -f {data_file} < {exported_csv_ref}\n\n")

    # make scripts executable
    try:
        os.chmod(export_path, 0o755)
        os.chmod(import_path, 0o755)
    except Exception:
        pass

    return export_path, import_path

# Example usage (uncomment and adjust path):
# generate_herc_scripts('/path/to/dir/with/conetic-files')

In [5]:
def main():
    target_dir = input("Enter target directory (containing conetic-files.txt): ").strip()
    if not target_dir:
        print("No directory provided. Exiting.")
        return
    try:
        export_script, import_script = generate_herc_scripts(target_dir)
        print("Generated:", export_script, import_script)
    except Exception as e:
        print("Error:", e)

if __name__ == '__main__':
    main()

Generated: /Users/kristoferpearson/Dev/Landy/Projects/FebRenSolsForSudha_20250930/001-HERC1-Export.sh /Users/kristoferpearson/Dev/Landy/Projects/FebRenSolsForSudha_20250930/002-HERC2-Import.sh
